In [51]:
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import StringType, LongType, NestedField
from pyiceberg.table import TableIdentifier
from pyiceberg.catalog.sql import SqlCatalog
# Load the catalog from the config file
import os

In [52]:
folder_path = 'sqllite_s3'

# Create folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

In [63]:
# Set AWS credentials via environment variables (optional if ~/.aws/credentials is set)
# os.environ["AWS_ACCESS_KEY_ID"] = "Qmhm2qgYBFEPTBE"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "@eaenaYgZtxfb&53"
# os.environ["AWS_ENDPOINT_URL"] = "https://p2-minio-dev.gates-staging.work/"
# Optional if using session token
# os.environ["AWS_SESSION_TOKEN"] = "your_session_token"
# os.environ["AWS_REGION"] = "us-east-1"

In [68]:
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{folder_path}/icyhot.db",
        "warehouse": f"s3://demobucket/warehouse",
        "s3.access-key-id": "CHANGEME",
        "s3.secret-access-key": "CHANGEME",
        "s3.endpoint": "CHANGEME",
        "s3.region": "us-east-1",
    },
)

In [70]:
catalog.create_namespace_if_not_exists('Sales')

In [71]:
from pyiceberg.schema import Schema
from pyiceberg.types import NestedField, IntegerType, StringType, FloatType, DoubleType, BooleanType
product_schema = Schema(
    NestedField(field_id=1, name='ProductId', field_type=IntegerType(), required=True),
    NestedField(field_id=2, name='Name', field_type=StringType(), required=True),
    NestedField(field_id=3, name='Category', field_type=StringType(), required=True),
    NestedField(field_id=4, name='Price', field_type=DoubleType(), required=True),
    NestedField(field_id=5, name='Stock', field_type=IntegerType(), required=True),
    NestedField(field_id=6, name='IsActive', field_type=BooleanType(), required=True)
)

In [72]:
product_table = catalog.create_table_if_not_exists(
    identifier="Sales.Product",
    schema=product_schema
)

In [73]:
import pyarrow as pa


product_data = pa.Table.from_pylist([
    {"ProductId": 1, "Name": "Laptop", "Category": "Electronics", "Price": 999.99, "Stock": 15, "IsActive": True},
    {"ProductId": 2, "Name": "Smartphone", "Category": "Electronics", "Price": 699.99, "Stock": 30, "IsActive": True},
    {"ProductId": 3, "Name": "Tablet", "Category": "Electronics", "Price": 399.99, "Stock": 20, "IsActive": True},
    {"ProductId": 4, "Name": "Smartwatch", "Category": "Wearable", "Price": 199.99, "Stock": 25, "IsActive": True},
    {"ProductId": 5, "Name": "Wireless Earbuds", "Category": "Accessories", "Price": 129.99, "Stock": 50, "IsActive": True},
    {"ProductId": 6, "Name": "Bluetooth Speaker", "Category": "Accessories", "Price": 89.99, "Stock": 35, "IsActive": True},
    {"ProductId": 7, "Name": "Gaming Console", "Category": "Gaming", "Price": 499.99, "Stock": 0, "IsActive": False},
    {"ProductId": 8, "Name": "Mechanical Keyboard", "Category": "Accessories", "Price": 79.99, "Stock": 40, "IsActive": True},
    {"ProductId": 9, "Name": "Gaming Mouse", "Category": "Accessories", "Price": 59.99, "Stock": 45, "IsActive": True},
    {"ProductId": 10, "Name": "Monitor", "Category": "Electronics", "Price": 249.99, "Stock": 20, "IsActive": True},
    {"ProductId": 11, "Name": "External Hard Drive", "Category": "Storage", "Price": 119.99, "Stock": 30, "IsActive": True},
    {"ProductId": 12, "Name": "USB Flash Drive", "Category": "Storage", "Price": 19.99, "Stock": 0, "IsActive": False},
    {"ProductId": 13, "Name": "Power Bank", "Category": "Accessories", "Price": 39.99, "Stock": 60, "IsActive": True},
    {"ProductId": 14, "Name": "VR Headset", "Category": "Gaming", "Price": 299.99, "Stock": 8, "IsActive": True},
    {"ProductId": 15, "Name": "Smart TV", "Category": "Electronics", "Price": 799.99, "Stock": 12, "IsActive": True},
    {"ProductId": 16, "Name": "Wireless Router", "Category": "Networking", "Price": 129.99, "Stock": 22, "IsActive": True},
    {"ProductId": 17, "Name": "Digital Camera", "Category": "Photography", "Price": 549.99, "Stock": 18, "IsActive": True},
    {"ProductId": 18, "Name": "Tripod Stand", "Category": "Photography", "Price": 49.99, "Stock": 0, "IsActive": False},
    {"ProductId": 19, "Name": "Fitness Tracker", "Category": "Wearable", "Price": 89.99, "Stock": 27, "IsActive": True},
    {"ProductId": 20, "Name": "Electric Toothbrush", "Category": "Health", "Price": 59.99, "Stock": 40, "IsActive": True},
    {"ProductId": 21, "Name": "Hair Dryer", "Category": "Personal Care", "Price": 39.99, "Stock": 28, "IsActive": True},
    {"ProductId": 22, "Name": "Air Purifier", "Category": "Home Appliances", "Price": 179.99, "Stock": 0, "IsActive": False},
    {"ProductId": 23, "Name": "Microwave Oven", "Category": "Home Appliances", "Price": 249.99, "Stock": 14, "IsActive": True},
    {"ProductId": 24, "Name": "Washing Machine", "Category": "Home Appliances", "Price": 499.99, "Stock": 7, "IsActive": True},
    {"ProductId": 25, "Name": "Refrigerator", "Category": "Home Appliances", "Price": 999.99, "Stock": 0, "IsActive": False}
], schema=product_table.schema().as_arrow())

In [74]:
product_table.append(df=product_data)

In [75]:
product_table.scan().select('ProductId', 'Price').to_arrow().to_pandas()

,ProductId,Price
0,1,999.99
1,2,699.99
2,3,399.99
3,4,199.99
4,5,129.99
5,6,89.99
6,7,499.99
7,8,79.99
8,9,59.99
9,10,249.99
